In [149]:
import requests
import pandas as pd
import json
from dateutil import parser

In [150]:
API_KEY = "897e9306b5860257fe7ad40a8db63a95-46cacd7e732372c7a5bcbdcc42bec0b6"
ACCOUNT_ID = "101-002-21354734-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [151]:
session = requests.Session()

In [152]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [153]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [154]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [155]:
response = session.get(url, params=None, data=None, headers=None)

In [156]:
response.status_code

200

In [157]:
data = response.json()

In [158]:
data

{'instruments': [{'name': 'XAU_GBP',
   'type': 'METAL',
   'displayName': 'Gold/GBP',
   'pipLocation': -2,
   'displayPrecision': 3,
   'tradeUnitsPrecision': 0,
   'minimumTradeSize': '1',
   'maximumTrailingStopDistance': '100.000',
   'minimumTrailingStopDistance': '0.050',
   'maximumPositionSize': '0',
   'maximumOrderUnits': '20000',
   'marginRate': '0.23',
   'guaranteedStopLossOrderMode': 'DISABLED',
   'tags': [{'type': 'ASSET_CLASS', 'name': 'COMMODITY'},
    {'type': 'KID_ASSET_CLASS', 'name': 'METAL'},
    {'type': 'BRAIN_ASSET_CLASS', 'name': 'METAL'}],
   'financing': {'longRate': '-0.0643',
    'shortRate': '0.0421',
    'financingDaysOfWeek': [{'dayOfWeek': 'MONDAY', 'daysCharged': 1},
     {'dayOfWeek': 'TUESDAY', 'daysCharged': 1},
     {'dayOfWeek': 'WEDNESDAY', 'daysCharged': 1},
     {'dayOfWeek': 'THURSDAY', 'daysCharged': 1},
     {'dayOfWeek': 'FRIDAY', 'daysCharged': 1},
     {'dayOfWeek': 'SATURDAY', 'daysCharged': 0},
     {'dayOfWeek': 'SUNDAY', 'daysChar

In [159]:
instruments_list = data['instruments']

In [160]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [161]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [162]:
instruments_dict = {}
for i in instruments_list:
    instruments_dict[i['name']] = {k: i[k] for k in key_i}

In [163]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [164]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [170]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )

    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
        return
    candles_df = get_candles_df(data)   
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")
    

In [172]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [173]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file("EUR_USD", count=4001, granularity=g)

EUR_USD H1 4000 candles, 2023-07-28 06:00:00+00:00 2024-03-20 21:00:00+00:00
EUR_USD H4 4000 candles, 2021-08-26 05:00:00+00:00 2024-03-20 17:00:00+00:00
EUR_USD H1 4000 candles, 2023-07-28 06:00:00+00:00 2024-03-20 21:00:00+00:00
EUR_USD H4 4000 candles, 2021-08-26 05:00:00+00:00 2024-03-20 17:00:00+00:00
EUR_USD H1 4000 candles, 2023-07-28 06:00:00+00:00 2024-03-20 21:00:00+00:00
EUR_USD H4 4000 candles, 2021-08-26 05:00:00+00:00 2024-03-20 17:00:00+00:00
EUR_USD H1 4000 candles, 2023-07-28 06:00:00+00:00 2024-03-20 21:00:00+00:00
EUR_USD H4 4000 candles, 2021-08-26 05:00:00+00:00 2024-03-20 17:00:00+00:00
EUR_USD H1 4000 candles, 2023-07-28 06:00:00+00:00 2024-03-20 21:00:00+00:00
EUR_USD H4 4000 candles, 2021-08-26 05:00:00+00:00 2024-03-20 17:00:00+00:00
EUR_USD H1 4000 candles, 2023-07-28 06:00:00+00:00 2024-03-20 21:00:00+00:00
EUR_USD H4 4000 candles, 2021-08-26 05:00:00+00:00 2024-03-20 17:00:00+00:00
EUR_USD H1 4000 candles, 2023-07-28 06:00:00+00:00 2024-03-20 21:00:00+00:00